# Importere data fra SSB med JSON-spørring

[Veiledning hos SSB](https://github.com/janbrus/ssb-api-python-examples/blob/master/jsonstatToPandas_funksjon.ipynb)

Koden her er hentet fra denne side for Tabell 10638


In [1]:
import json
import pandas as pd
import requests
# pyjstat er biblioteket for å lese JSON-stat, 'pip install pyjstat' 
from pyjstat import pyjstat

## Alternativ 1 - Lim inn POST Url og JSON-spørring

In [2]:
# Lim in PostUrl som spørringen skal postes mot
postUrl = "https://data.ssb.no/api/v0/no/table/10638/"


In [4]:
# Spørringen og endepunktet til API-et hentes fra Statistikkbanken eller API-konsoll. 
# apiQuery = # lim inn spørringen din her


apiQuery = {
  "query": [
    {
      "code": "KonsumInnd",
      "selection": {
        "filter": "item",
        "values": [
          "TOTK"
        ]
      }
    },
    {
      "code": "Reiseliv",
      "selection": {
        "filter": "item",
        "values": [
          "0",
          "01",
          "01.01",
          "01.02",
          "01.03",
          "01.04",
          "01.05",
          "01.06",
          "01.07",
          "01.08",
          "01.09",
          "01.10",
          "02"
        ]
      }
    },
    {
      "code": "ContentsCode",
      "selection": {
        "filter": "item",
        "values": [
          "FastePriser"
        ]
      }
    }
  ],
  "response": {
    "format": "json-stat2"
  }
}

In [5]:
# Funksjon for å konvertere uttrekket til 2 dataframes, tar 2 paramtre

def apiToDataframe(postUrl, query):

    # postUrl som spørringen skal postes mot
    # Spørringen og endepunktet til API-et kan hentes fra Statistikkbanken.

    res = requests.post(postUrl, json=query)
    # legger resultat i ds. DS har i tillegg en del metadata
    ds = pyjstat.Dataset.read(res.text)
    # skriver resultatet til to dataframes
    # først dataframe med tekst
    df = ds.write('dataframe')
    # deretter dataframe med koder
    df_id = ds.write('dataframe', naming='id')
    # returnerer også ds i tilfelle en trenger metadata
    return df, df_id, ds

In [6]:
df, df_id, ds = apiToDataframe(postUrl, apiQuery)

In [7]:
df

,konsumentgruppe,reiselivsnæring,statistikkvariabel,år,value
0,Totalt konsum,Turistkonsum i alt,Faste 2020-priser (mill. kr),2007,155191.0
1,Totalt konsum,Turistkonsum i alt,Faste 2020-priser (mill. kr),2008,155837.0
2,Totalt konsum,Turistkonsum i alt,Faste 2020-priser (mill. kr),2009,149560.0
3,Totalt konsum,Turistkonsum i alt,Faste 2020-priser (mill. kr),2010,153775.0
4,Totalt konsum,Turistkonsum i alt,Faste 2020-priser (mill. kr),2011,158573.0
...,...,...,...,...,...
190,Totalt konsum,Turistkonsum utenom reiselivsprodukter,Faste 2020-priser (mill. kr),2017,65241.0
191,Totalt konsum,Turistkonsum utenom reiselivsprodukter,Faste 2020-priser (mill. kr),2018,68070.0
192,Totalt konsum,Turistkonsum utenom reiselivsprodukter,Faste 2020-priser (mill. kr),2019,68877.0
193,Totalt konsum,Turistkonsum utenom reiselivsprodukter,Faste 2020-priser (mill. kr),2020,65188.0


## Alternativ 2 Lag en funksjon ("jsqToTab") for å hente data via PxWebApi fra en lagret API-spørring på fil

Etter at du har laget tabellen hos SSB trykk "Lagre API-spørring", og legg fila i samme katalog som du har Jupyter notatblokka.

In [8]:
import os

import urllib.request

urllib.request.urlretrieve('https://raw.githubusercontent.com/uit-sok-1006-v24/uit-sok-1006-v24.github.io/main/ssbapi_table_10638.json', 
                'ssbapi_table_10638.json')


savedApiQuery = json.load(open('ssbapi_table_10638.json'))


In [9]:
# Funksjonen tar 1-2 paramtre
# Leser filer på formen ssbapitable_03013.json, der 03013 er ID til tabellen. 


# LEGG MERKE TIL NESTE SETNING!
# Eksempelet forutsetter at filen med API-spørring ligger i samme katalog.
# DVS SAMME PLASS SOM DU LAGRER JUPYTER NOTATBLOKKEN DIN


# funksjon som tar id på lagret API-spørring som input
# lang kan angi språk, default er norsk
# funksjonen returnerer to dataframes, en med tekst og en med koder

def jsqToTab(qid, lang='no'):
    
    savedApiQuery = json.load(open('ssbapi_table_' + qid + '.json'))
    # Vi finner spørringen og Tabell ID
    apiQuery = savedApiQuery['queryObj']
    tabId = savedApiQuery['tableIdForQuery']
    # PostUrl som spørringen skal postes mot
    # Spørringen og endepunktet til API-et kan hentes fra Statistikkbanken.
    postUrl = "https://data.ssb.no/api/v0/" + lang + "/table/" + tabId
    res = requests.post(postUrl, json=apiQuery)
    # legger resultat i ds
    ds = pyjstat.Dataset.read(res.text)
    # skriver resultatet til to dataframes
    df_id = ds.write('dataframe', naming='id')
    df = ds.write('dataframe')

    return df, df_id

In [10]:
df, df_id = jsqToTab("10638") #sett inn 5-sifret tabellID fra filnavn f.eks. 03013 slik jsqToTab("03013")

In [11]:
df

,konsumentgruppe,reiselivsnæring,statistikkvariabel,år,value
0,Totalt konsum,Turistkonsum i alt,Faste 2020-priser (mill. kr),2007,155191.0
1,Totalt konsum,Turistkonsum i alt,Faste 2020-priser (mill. kr),2008,155837.0
2,Totalt konsum,Turistkonsum i alt,Faste 2020-priser (mill. kr),2009,149560.0
3,Totalt konsum,Turistkonsum i alt,Faste 2020-priser (mill. kr),2010,153775.0
4,Totalt konsum,Turistkonsum i alt,Faste 2020-priser (mill. kr),2011,158573.0
...,...,...,...,...,...
190,Totalt konsum,Turistkonsum utenom reiselivsprodukter,Faste 2020-priser (mill. kr),2017,65241.0
191,Totalt konsum,Turistkonsum utenom reiselivsprodukter,Faste 2020-priser (mill. kr),2018,68070.0
192,Totalt konsum,Turistkonsum utenom reiselivsprodukter,Faste 2020-priser (mill. kr),2019,68877.0
193,Totalt konsum,Turistkonsum utenom reiselivsprodukter,Faste 2020-priser (mill. kr),2020,65188.0


In [12]:
df_id

,KonsumInnd,Reiseliv,ContentsCode,Tid,value
0,TOTK,0,FastePriser,2007,155191.0
1,TOTK,0,FastePriser,2008,155837.0
2,TOTK,0,FastePriser,2009,149560.0
3,TOTK,0,FastePriser,2010,153775.0
4,TOTK,0,FastePriser,2011,158573.0
...,...,...,...,...,...
190,TOTK,02,FastePriser,2017,65241.0
191,TOTK,02,FastePriser,2018,68070.0
192,TOTK,02,FastePriser,2019,68877.0
193,TOTK,02,FastePriser,2020,65188.0


In [13]:
df1, df_id1 = jsqToTab("10638", 'en') #sett inn 5 sifret tabellID 

In [14]:
df1

,consumer group,tourism industry,contents,year,value
0,Total tourism consumption,Total consumption expenditures by tourists,Constant 2020 prices (NOK million),2007,155191.0
1,Total tourism consumption,Total consumption expenditures by tourists,Constant 2020 prices (NOK million),2008,155837.0
2,Total tourism consumption,Total consumption expenditures by tourists,Constant 2020 prices (NOK million),2009,149560.0
3,Total tourism consumption,Total consumption expenditures by tourists,Constant 2020 prices (NOK million),2010,153775.0
4,Total tourism consumption,Total consumption expenditures by tourists,Constant 2020 prices (NOK million),2011,158573.0
...,...,...,...,...,...
190,Total tourism consumption,Total tourism consumption of other products,Constant 2020 prices (NOK million),2017,65241.0
191,Total tourism consumption,Total tourism consumption of other products,Constant 2020 prices (NOK million),2018,68070.0
192,Total tourism consumption,Total tourism consumption of other products,Constant 2020 prices (NOK million),2019,68877.0
193,Total tourism consumption,Total tourism consumption of other products,Constant 2020 prices (NOK million),2020,65188.0


In [15]:
df_id1

,KonsumInnd,Reiseliv,ContentsCode,Tid,value
0,TOTK,0,FastePriser,2007,155191.0
1,TOTK,0,FastePriser,2008,155837.0
2,TOTK,0,FastePriser,2009,149560.0
3,TOTK,0,FastePriser,2010,153775.0
4,TOTK,0,FastePriser,2011,158573.0
...,...,...,...,...,...
190,TOTK,02,FastePriser,2017,65241.0
191,TOTK,02,FastePriser,2018,68070.0
192,TOTK,02,FastePriser,2019,68877.0
193,TOTK,02,FastePriser,2020,65188.0


# Nedlasting
Dersom du har ei fil som er på GitHub eller på din maskin i en annen katalog kan følgende kode være nyttig. Fra GitHub bruker du "raw" versjonen av fila. Min fil er her: https://raw.githubusercontent.com/uit-sok-1006-v24/uit-sok-1006-v24.github.io/main/ssbapi_table_10638.json 

In [16]:
import os

import urllib.request


In [17]:
def jsqToTab_remote(qid, lang='no'):
    urllib.request.urlretrieve('https://raw.githubusercontent.com/uit-sok-1006-v24/uit-sok-1006-v24.github.io/main/ssbapi_table_10638.json', 
                'ssbapi_table_10638.json')
    savedApiQuery = json.load(open('ssbapi_table_10638.json'))
    # Vi finner spørringen og Tabell ID
    apiQuery = savedApiQuery['queryObj']
    tabId = savedApiQuery['tableIdForQuery']
    # PostUrl som spørringen skal postes mot
    # Spørringen og endepunktet til API-et kan hentes fra Statistikkbanken.
    postUrl = "https://data.ssb.no/api/v0/" + lang + "/table/" + tabId
    res = requests.post(postUrl, json=apiQuery)
    # legger resultat i ds
    ds = pyjstat.Dataset.read(res.text)
    # skriver resultatet til to dataframes
    df_id = ds.write('dataframe', naming='id')
    df = ds.write('dataframe')

    return df, df_id

In [18]:
df, df_id = jsqToTab_remote("10638") 

In [19]:
df

,konsumentgruppe,reiselivsnæring,statistikkvariabel,år,value
0,Totalt konsum,Turistkonsum i alt,Faste 2020-priser (mill. kr),2007,155191.0
1,Totalt konsum,Turistkonsum i alt,Faste 2020-priser (mill. kr),2008,155837.0
2,Totalt konsum,Turistkonsum i alt,Faste 2020-priser (mill. kr),2009,149560.0
3,Totalt konsum,Turistkonsum i alt,Faste 2020-priser (mill. kr),2010,153775.0
4,Totalt konsum,Turistkonsum i alt,Faste 2020-priser (mill. kr),2011,158573.0
...,...,...,...,...,...
190,Totalt konsum,Turistkonsum utenom reiselivsprodukter,Faste 2020-priser (mill. kr),2017,65241.0
191,Totalt konsum,Turistkonsum utenom reiselivsprodukter,Faste 2020-priser (mill. kr),2018,68070.0
192,Totalt konsum,Turistkonsum utenom reiselivsprodukter,Faste 2020-priser (mill. kr),2019,68877.0
193,Totalt konsum,Turistkonsum utenom reiselivsprodukter,Faste 2020-priser (mill. kr),2020,65188.0
